In [52]:
import os
import face_recognition
import numpy
import cv2
from typing import List

In [74]:
IMAGE_FOLDER = 'images'
WIDTH, HEIGHT = 1200, 800
GAUSSIAN_KERNEL_SIZE = (345, 345)
X_STANDARD_DEVIATION = 0

In [98]:
def get_image_paths_and_names(folder_path: str) -> List[str]:
    image_path_list, image_name_list = [], []
    with os.scandir(folder_path) as entries:
        for entry in entries:
            if entry.is_file() and entry.name.endswith('jpg'):
                image_path = os.path.join(folder_path, entry.name)
                image_path_list.append(image_path)
                image_name_list.append(entry.name)
    return image_path_list, image_name_list

def load_images(image_path_list: List[str]) -> numpy.ndarray:
    return [cv2.imread(image_path) for image_path in image_path_list]

# Detect faces
def resize_images(image_list: List[numpy.ndarray]) -> List[numpy.ndarray]:
    return [cv2.resize(image, (WIDTH, HEIGHT)) for image in image_list]

def rescale_face_boxes(face_boxes, shape):
    rescaled_faces = [
        [int(top * shape[0] / HEIGHT), int(right * shape[1] / WIDTH),
        int(bottom * shape[0] / HEIGHT), int(left * shape[1] / WIDTH)] \
        for top, right, bottom, left in face_boxes]
    return rescaled_faces

def detect_faces_in_images(image_list):
    resized_image_list = resize_images(image_list)
    rescaled_face_boxes_batch = face_recognition.batch_face_locations(
        resized_image_list, number_of_times_to_upsample=1)
    #face_boxes_batch = [face_recognition.face_locations(resized_image) for resized_image in resized_image_list]
    image_shape_list = [image.shape for image in image_list]
    face_boxes_batch = [rescale_face_boxes(face_boxes, image_shape) for face_boxes, image_shape in
                            zip(rescaled_face_boxes_batch, image_shape_list)]
    return face_boxes_batch


# Blur faces
def crop_faces(image: numpy.ndarray, face_boxes) -> List[numpy.ndarray]:
    return [image[top:bottom, left:right] for top, right, bottom, left in face_boxes]

def blur_faces_in_image(raw_image: numpy.ndarray, face_crops: List[numpy.ndarray], face_boxes) -> numpy.ndarray:
    blurred_image = raw_image.copy()
    blurred_faces = [cv2.GaussianBlur(face_crop, GAUSSIAN_KERNEL_SIZE, X_STANDARD_DEVIATION) \
        for face_crop in face_crops]
    for blurred_face, (top, right, bottom, left) in zip(blurred_faces, face_boxes):
        blurred_image[top:bottom, left:right] = blurred_face
    return blurred_image

def blur_faces_in_images(raw_image_batch: List[numpy.ndarray], face_boxes_batch) -> List[numpy.ndarray]:
    blurred_raw_image_batch = []
    for raw_image, face_boxes in zip(raw_image_batch, face_boxes_batch):
        if len(face_boxes) > 0:
            face_crops = crop_faces(raw_image, face_boxes)
            blurred_image = blur_faces_in_image(raw_image, face_crops, face_boxes)
        else:
            blurred_image = raw_image
        blurred_raw_image_batch.append(blurred_image)
    return blurred_raw_image_batch

# Save images
def save_images(image_list, image_name_list, folder):
    for image, image_name in zip(image_list, image_name_list):
        new_path = os.path.join(folder, f'blurred_{image_name}')
        print(new_path)
        is_saved = cv2.imwrite(new_path, image)

In [99]:
image_path_list, image_name_list = get_image_paths_and_names(IMAGE_FOLDER)
image_list = load_images(image_path_list)
face_boxes_batch = detect_faces_in_images(image_list)
blurred_images = blur_faces_in_images(image_list, face_boxes_batch)
save_images(blurred_images, image_name_list, IMAGE_FOLDER)

images/blurred_2.jpg
images/blurred_3.jpg
images/blurred_1.jpg
